<a href="https://colab.research.google.com/github/YangchenHuang/Business_Analytics/blob/master/Assignments/Yangchen_Huang_HW3_(IEOR_E4650_Spring_2020_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IEOR 4650  Business Analytics (Spring 2020)**

**Homework 3 [Total points: 60]**

**Due: 11:59 PM, Feb 26**

Homework 3 consists of Python programming practices and the applications to 
* Time series
* Maximum Likelihood Estimation

The steps to finish the homework:

*   Step 1: Answer your homework on Google Colab.
*   Step 2: Submit the copy on Google Assignment

Before you submit your assignment, make sure to re-run your code from the beginning. (You can do so by first clicking Runtime/Reset All Runtimes and then clicking Runtime/Run all)

In [0]:
#This imports all the packages you need for the homework
#Please run this first


!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


#import modules

import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from statsmodels.base.model import GenericLikelihoodModel
import scipy.special as spsp


#Exercise 1 [18 Points]
In this exercise, let's continue working with a dataset which explores the store sales.

Before doing the assignment, please study this dataset [here](https://www.kaggle.com/c/rossmann-store-sales/data)


For this study, we are interested in finding a good model that gives good prediction performance. 

* Especially, we use RMSPE as our accuracy measurement. RMSPE is defined as following:
$$RMSPE = \sqrt{\frac{1}{{N}}{(\frac{y_i-\widehat{y_i}}{y_i})^2}}$$
* In addition, any day and store with 0 actual sales is ignored in scoring.

In [2]:
#Load the dataset
link="https://drive.google.com/open?id=1Q6J0q4tlWJ7TajX6hMSvZ8tK3CyDD8QY"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Sales.csv')  
SALE = pd.read_csv('Sales.csv')
SALE.head(5)



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


*You might see a warning when importing the data. This is because StateHoliday has both values in string and numbers. No worries this for now.*

In [3]:
link="https://drive.google.com/open?id=1g6URzUJnhCLOtg0a9TTHC18KYcqwxvoc"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('myfile_store.csv')  
STORE = pd.read_csv('myfile_store.csv')
STORE.head(5)

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


## Q1 [3 points]
**[Data Cleaning]**

For this questions, we continue from the last exercise. The dataset has been partially cleaned.

For this data, let's create some additional variables to allow us incorporate more richness to the model. 


* For each observation, create the variable "Sales_lag1" that gives the number of sales from the previous positive-sales day for this store.
* For each observation, create the variable "Sales_lag2" that gives the number of sales from two periods ago.
* For each observation, create the variable "Customer_lag1" that gives the number of customers from the previous positive-sales day for this store.
* For each observation, create the variable "Customer_lag2" that gives the number of sales from two periods ago.



In [0]:
#These lines of command finish the data cleaning from the last assignment
SALE=SALE.merge(STORE)  
SALE=SALE.astype({"Date":"datetime64"}) 
SALE=SALE[SALE["Sales"]>0]  
SALE=SALE[(SALE["Store"]>=1) & (SALE["Store"]<=100)]  
SALE=SALE.assign(StateHoliday= lambda x: x.StateHoliday.replace(0,"0"))
SALE=SALE.astype({"Date":"datetime64"})
SALE=SALE.assign(Month=lambda x: x.Date.dt.month)
basedate = pd.Timestamp('2013-01-01')
SALE=SALE.assign(Sales_lag1=lambda x: x.Sales.shift(periods=1))
SALE=SALE.assign(Sales_lag2=lambda x: x.Sales.shift(periods=2))
SALE=SALE.assign(Customer_lag1=lambda x: x.Customers.shift(periods=1))
SALE=SALE.assign(Customer_lag2=lambda x: x.Customers.shift(periods=2))


#Add commands for this question

In [8]:
SALE

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Month,Sales_lag1,Sales_lag2,Customer_lag1,Customer_lag2
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,7,5263.0,NaN,555.0,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,7,5020.0,5263.0,546.0,555.0
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,7,4782.0,5020.0,523.0,546.0
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,7,5011.0,4782.0,560.0,523.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75586,100,1,2013-01-07,10609,823,1,1,0,0,d,a,17930.0,NaN,NaN,0,NaN,NaN,NaN,1,6816.0,7883.0,611.0,645.0
75587,100,6,2013-01-05,7973,681,1,0,0,0,d,a,17930.0,NaN,NaN,0,NaN,NaN,NaN,1,10609.0,6816.0,823.0,611.0
75588,100,5,2013-01-04,7129,620,1,0,0,1,d,a,17930.0,NaN,NaN,0,NaN,NaN,NaN,1,7973.0,10609.0,681.0,823.0
75589,100,4,2013-01-03,7398,672,1,0,0,1,d,a,17930.0,NaN,NaN,0,NaN,NaN,NaN,1,7129.0,7973.0,620.0,681.0


## Q2 [12 points]

Again, we split the data into the following segments

(1) Training: Time window 2013-01-01 to 2015-03-31  

(2) Validation: Time window 2015-04-01 to 2015-05-31 

(3) Testing: Time window 2015-06-01 to 2015-07-31

For this question, let's try to construct a model that can help us predict the sales for the next day.

* Regressing log(Sales) on: log(Sales\_lag1),  log(Customers\_lag1), C(DayOfWeek), C(Month), Promo, StateHoliday, SchoolHoliday, C(Store) [Model 1]
* Use RMSPE to evaluate the performance on the the validation set. [Again, the focus is how to accurately predict the sales not log(sales) ]
* Show whether adding log(Sales\_lag2), and log(Customers\_lag2) to this model can improve model fit. [Model 2]
* Based on Model1, use line plot to show the model fitting by having
>>* a line of DaysSinceFirst (x-axis) v.s. actual sales (y-axis)  for  points in the validation set for store=50 [use blue color]
>>* a line of DaysSinceFirst (x-axis) v.s. predicted sales (y-axis)  for  points in the validation set for store=50 [use green color]
>>* Add a legend to the plot to indicate what each line indicates.

In [0]:
#These lines of command finish the data split from the last assignment
SALE=SALE.assign(DaysSinceFirst=lambda x: (x.Date-basedate).dt.days)
SALE_Training=SALE[SALE["Date"]<="2015-03-31"]
SALE_Validation=SALE[(SALE["Date"]>="2015-04-01") & (SALE["Date"]<="2015-05-31")]
SALE_Testing=SALE[SALE["Date"]>="2015-06-01"]

#Add additional commands
model1=ols(formula="np.log(Sales)~np.log(Sales_lag1)+np.log(Customer_lag1)+C(DayOfWeek)+C(Month)+Promo+StateHoliday+SchoolHoliday+C(Store)", data=SALE_Training).fit()
def rmspe(y_pred, y_true):
    loss = np.sqrt(np.mean(np.square(((y_true - y_pred) / y_true))))
    return loss
rmspe(model1.predict(SALE_Validation),SALE_Validation["Sales"])
print(model1.summary())

## Q3 [3 Points]

Now, let's assume that the company wants to do a two-day ahead prediction. Let's run the following model

* regressing log(Sales) on:  log(Sales\_lag2), log(Customers\_lag2), C(DayOfWeek), C(Month), Promo, StateHoliday, SchoolHoliday, C(Store)
* Use RMSPE to evaluate the performance on the validation set. [Again, the focus is how to accurately predict the sales not log(sales) ]
* Compare with one-day ahead prediction, see how model fitting changes.

# Exercise 2 [42 Points]

In this exercise, let's take a deeper look at Poisson model

This data measures the number of ad exposures each customer got **within 1 day**. 


In [0]:
#Load the dataset
link="https://drive.google.com/open?id=1pjdoOKuczV8Zc6Zw8-yUh6q2tqogN-xt"
_,id=link.split("=")
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Exposure.csv')  
Ad = pd.read_csv('Exposure.csv')
Ad.head(5)



,Cookie_ID,Exposures
0,1,0
1,2,0
2,3,3
3,4,49
4,5,4


## Q1 [2 points]

For this exercise, let's first separate the data based on sklearn.model_selection

* Split the data to training (90% of the samples) and validation set (10% of the samples), with random_state=42
* Print the average of outcomes in the validation set.


##Q2 [10 Points]
We want to estimate a one-segment Poisson model based on the training set

* Define a function that returns the joint-log-likelihood assuming that the outcomes are sampled from a Poisson distribution. Again, remember to assume that $\lambda=exp(\beta)$

* Implement a naive search to find the best $\beta$ by looping through every possible $\beta$ value inside `np.arange(-1,2,0.001)`

* The closed solution for an MLE poisson problem has $\hat{\lambda}=\bar{y}$, where $y$ is a vector of all the count outcomes. Compare the best $\hat{\lambda}$ from the naive search with the closed-form solution.
* Assume that when increasing the exposure length to $a$ units of time, the number of exposures received by each customer will still follow a Poisson distribution with new $\lambda'=a\lambda$. Based on the model, how is the size of customers who did not receive an exposure within 1 day expected to change if we increase the length of exposures to 4 days? For $\hat{\lambda}$, use the result based on the closed-form solution.

In [0]:
#Define Likelihood function

In [0]:
#Perform naive search

In [0]:
#Compare naive search with closed-form solution

In [0]:
#how is the size of customers who did not receive a exposures expected to change 
#if we increase the length of exposures to 4 days?

## Q3 [12 Points]

For a two-segment model, we did not put a constraint on the relationship between $\lambda_1$ (Segment1) and $\lambda_2$ (Segment2) in the lecture. As a result, certain iterations will have $\lambda_1>\lambda_2$, while certain iterations will have $\lambda_1>\lambda_2$.

* Think about a way to constrain the parameters for the two-segment model to always give $\lambda_1<\lambda_2$ when setting starting_params=np.random.rand(3). Implement this algorithm and report $\hat{\lambda_1}$ and $\hat{\lambda_2}$. 

* Based on this model, how many people do you predict on the validation set to have at least 2 exposures? How does this number compare to the actual numbers?

* If a consumer belongs to segment 1, what is the probability of this customer receiving at least 2 exposures? What if this customer belongs to segment2?

* If a customer received at least 2 exposures, what is the probability for this customer to belong to segment 1? How about segment2?

In [0]:
class Poisson_Mix2(GenericLikelihoodModel):



In [0]:
#Based on this model, how many people do you predict on the validation set to have
# at least 2 exposures? How does this number compare to the actual numbers?








In [0]:
#If a consumer belongs to segment 1, what is the probability of this customer 
#recieving at least 2 advertisement exposures? What if this customer belongs to segment2?


In [0]:
#If a customer received at least 2 advertisement exposures, what is the probability
# for this customer to belong to segment 1? How about segment2?



##Q4 [12 Points]

Now run an NBD model on the validation set and answer the following questions:

* Based on this model, how many people in the validation set do you predict  to have at least 2 exposures? How different is this number from the actual numbers?

* If a customer received 3 advertisement exposures, plot the posterior distribution of the $\lambda$ for this customer.

* For all the customers receiving 3 advertisement exposures in the validation set, what is the expected number of customers receiving at least 2 advertisement exposures in the next period?




In [0]:
class NBD(GenericLikelihoodModel):
  

In [0]:
#Based on this model, how many people in the validation set do you predict to have at least 2 exposures?
# How does this number compare to the actual numbers?

In [0]:
#If a customer received 3 advertisement exposures,
# plot the posterior distribution of the  λ  for this customer.

In [0]:
#For all the customers receiving 3 advertisment exposures in the validation set, 
#what is the expected number of customers receiving at least 2 advertisement exposures in the next period?

## Q5 [6 Points]

Compare the model in Q3 and Q4 and use the following 3 measurements to discuss which model is better

* AIC of the model on the training set
* BIC of the model on the training set
* Joint log-likelihood for data points on the validation set